<a href="https://colab.research.google.com/github/sobelucad9/classificateur_vetements/blob/main/CPA_Attack_sur_AES_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CPA Attack sur AES-128**
Ce code implémente une **attaque CPA (Correlation Power Analysis)** sur **AES-128** pour récupérer la clé secrète en utilisant des traces de consommation.


### **1. Importation des bibliothèques**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### **2. Chargement des données**

In [2]:
plaintexts = np.load('/content/sample_data/textin_array.npy')  # (num_traces, 16)
traces = np.load('/content/sample_data/trace_array.npy')      # (num_traces, num_samples)

**Explication :**  
- **`plaintexts`** contient les **textes clairs** utilisés dans AES (format `(num_traces, 16)`, 16 bytes par texte clair).  
- **`traces`** contient les **traces de puissance** mesurées pendant l'exécution de l'AES (`num_traces x num_samples`).  


### **3. Définition de la table S-Box AES**

In [4]:
sbox = np.array([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
                 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
                 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
                 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
                 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
                 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
                 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
                 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
                 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
                 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
                 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
                 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
                 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
                 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
                 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
                 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
                 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
                 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
                 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
                 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
                 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
                 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
                 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
                 0x54, 0xbb, 0x16]
                , dtype=np.uint8)

### **4. Calcul du poids de Hamming (Hamming Weight)**

In [5]:
hw = np.array([bin(x).count('1') for x in range(256)])

**Explication :**  
- Le **modèle de puissance CPA** suppose que la consommation dépend du **nombre de bits à 1** (poids de Hamming).  
- Cette ligne pré-calcule le **poids de Hamming pour tous les octets possibles (0-255)**.


### **5. Initialisation des variables**

In [6]:
num_traces, num_samples = traces.shape
key_guess = np.zeros(16, dtype=np.uint8)

**Explication :**  
- `num_traces` = nombre de traces AES disponibles.  
- `num_samples` = nombre d'échantillons dans une trace.  
- `key_guess` = tableau pour stocker les **16 bytes de la clé AES** trouvés.

### **6. Attaque CPA - Deviner chaque octet de la clé**

In [7]:
for key_byte in tqdm(range(16), desc="Attacking key bytes"):
    # Extract plaintext bytes for the current key byte position
    pt_byte = plaintexts[:, key_byte]

    # Generate all possible key guesses (0-255)
    guesses = np.arange(256, dtype=np.uint16)

    # Compute intermediate values (S-Box output) for all traces and guesses
    xor = np.bitwise_xor(pt_byte[:, np.newaxis], guesses[np.newaxis, :])
    intermediate = sbox[xor]

    # Compute Hamming weights of intermediates
    hypo_hw = hw[intermediate]

    # Normalize hypothetical power values
    hypo_mean = np.mean(hypo_hw, axis=0)
    hypo_std = np.std(hypo_hw, axis=0)
    hypo_std[hypo_std == 0] = 1e-12  # Avoid division by zero
    hypo_norm = (hypo_hw - hypo_mean) / hypo_std

    # Normalize traces
    traces_mean = np.mean(traces, axis=0)
    traces_std = np.std(traces, axis=0)
    traces_std[traces_std == 0] = 1e-12
    traces_norm = (traces - traces_mean) / traces_std

    # Compute correlation matrix (256 guesses x num_samples)
    corr_matrix = np.dot(hypo_norm.T, traces_norm) / (num_traces - 1)

    # Find the guess with the highest maximum correlation
    max_corrs = np.max(np.abs(corr_matrix), axis=1)
    best_guess = np.argmax(max_corrs)
    key_guess[key_byte] = best_guess

Attacking key bytes: 100%|██████████| 16/16 [00:04<00:00,  3.89it/s]


### **7. Affichage de la clé récupérée**


In [8]:
print("Recovered key:", [hex(x) for x in key_guess])

Recovered key: ['0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c']


### **8. Vérification avec la clé réelle (si disponible)**

In [9]:
try:
    known_key = np.load('/content/sample_data/known_keys.npy').flatten()
    print("Known key:", [hex(x) for x in known_key])
    if np.array_equal(key_guess, known_key):
        print("Key matches!")
    else:
        print("Key does not match.")
except FileNotFoundError:
    print("No known_keys.npy found for verification.")


Known key: ['0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c', '0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x